In [2]:
from datasets import load_dataset
import nltk
nltk.download('punkt')


model_name = "facebook/bart-large-cnn"
dataset_name = "kmfoda/booksum"

# Load the dataset
dataset = load_dataset(dataset_name)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emmanuelmoran\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,  DataCollatorForSeq2Seq

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [10]:
from transformers import BartTokenizer

max_length = 4096

def preprocess_data(examples):
    inputs = [ f'summarize the following chapter: {example}\nsummary:' for example in examples['chapter']]
    targets = examples['summary']
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=max_length, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1484 [00:00<?, ? examples/s]

Map:   0%|          | 0/1431 [00:00<?, ? examples/s]

In [11]:
from statistics import mean
print(mean(dataset['train']['chapter_length']))
print(max(dataset['train']['chapter_length']))
print(min(dataset['train']['chapter_length']))

3897.230625
114226.0
42.0


In [12]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-booksum-pgc",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)


d:\OTROS\LLMs\llm-venv\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [17]:
trainer.train()

  0%|          | 0/12000 [00:00<?, ?it/s]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
